In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from magpylib import magnet
from nn_magnetics.utils.physics import (
    Dz_cuboid_elementwise,
    Dz_cuboid,
    Bfield_homogeneous,
)
from nn_magnetics.utils.metrics import angle_error, relative_amplitude_error, calculate_metrics_baseline
from nn_magnetics.data.create_data import generate_points_grid
import numpy as np
from nn_magnetics.data import AnisotropicData
from nn_magnetics.models import QuaternionNet, AngleAmpCorrectionNetwork
from pathlib import Path
import torch
from nn_magnetics.utils.plotting import plot_heatmaps
from time import perf_counter
from nn_magnetics.predictions import B as calcB
import magpylib
from magpylib_material_response import meshing, demag

In [3]:
torch.set_printoptions(precision=8)

In [4]:
valid_data = AnisotropicData("../data/3dof_chi_v2/validation_fast", device="cpu")
X, B = valid_data.get_magnets()
X, B = X.reshape(-1, 8), B.reshape(-1, 6)

In [5]:
n_samples = X.shape[0]
aa, bb = X[:,0].unsqueeze(-1), X[:,1].unsqueeze(-1)
chix, chiy, chiz = X[:,2], X[:,3], X[:,4]

grid_normalized = X[:, 5:]
grid = grid_normalized.clone().requires_grad_(True)
# grid[:, 0] *= a
# grid[:, 1] *= b
_, B_reduced = B[:,:3], B[:,3:]

In [6]:
# aa = torch.ones((n_samples, 1)) * a
# bb = torch.ones((n_samples, 1)) * b
cc = torch.ones((n_samples, 1))

dims = torch.concat((aa, bb, cc), dim=1)

In [ ]:



pz = 1 / (1 + chiz * Dz_cuboid(dims)).unsqueeze(-1)

polarizations = torch.concat((torch.zeros((n_samples, 1)),torch.zeros((n_samples, 1)),pz), dim=1)

B_torch = Bfield_homogeneous(
    observers=grid,
    dimensions=dims,
    polarizations=polarizations,
)

div = torch.zeros(n_samples)  # Store divergence for each batch
    
for i in range(3):  # Loop over Bx, By, Bz
        grad_Bi = torch.autograd.grad(B_torch[:, i], grid, grad_outputs=torch.ones_like(B_torch[:, i]), retain_graph=True)[0]
        div += grad_Bi[:, i] 

In [16]:
div.min()

tensor(-2.42016972e-12)

In [ ]:
torch.mean(angle_error(B_torch, B_reduced)), torch.mean(relative_amplitude_error(B_torch, B_reduced, False))

(tensor(2.91658852e-07), tensor(0.))